In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

import fates_calibration_library.analysis_functions as analysis
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.plotting_functions as plotting
import fates_calibration_library.utils as utils

In [2]:
import importlib
import fates_calibration_library.analysis_functions
importlib.reload(fates_calibration_library.analysis_functions)

<module 'fates_calibration_library.analysis_functions' from '/glade/work/afoster/FATES_calibration/fates_calibration_library/fates_calibration_library/analysis_functions.py'>

In [3]:
# Setup PBSCluster
cluster = PBSCluster(
    cores=1,                                                   # The number of cores you want
    memory='25GB',                                             # Amount of memory
    processes=1,                                               # How many processes
    queue='casper',                                            # The type of queue to utilize
    local_directory='/glade/work/afoster',                     # Use your local directory
    resource_spec='select=1:ncpus=1:mem=25GB',                 # Specify resources
    log_directory='/glade/derecho/scratch/afoster/dask_logs',  # log directory
    account='P08010000',                                       # Input your project ID here
    walltime='04:00:00',                                       # Amount of wall time
    interface='ext')

/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45241 instead
  warnings.warn(


In [4]:
cluster.scale(30)
dask.config.set({
    'distributed.dashboard.link': 'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'
})
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/45241/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/45241/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.182:45791,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/afoster/proxy/45241/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
fates_vars = ['FATES_FRACTION', 'FATES_GPP', 'FATES_LAI', 
             'QVEGE', 'QVEGT', 'QSOIL', 'EFLX_LH_TOT', 'FSH', 
             'QRUNOFF', 'RAIN', 'SNOW', 'H2OSNO', 'TSOI',
             'SOILWATER_10CM', 'FSR', 'FSDS', 'FSA', 'FIRE', 'FLDS', 'FGR12',
             'TSA', 'TREFMXAV', 'TREFMNAV', 'RH2M']

In [6]:
# sparse grid clustering file
grid_dir = '/glade/work/afoster/FATES_calibration/surfdata'
grid_file = 'clusters.clm51_PPEn02ctsm51d021_2deg_GSWP3V1_leafbiomassesai_PPE3_hist.annual+sd.400.nc'
sparse_grid = xr.open_dataset(os.path.join(grid_dir, grid_file))

# sparse grid whittaker biomes
whittaker_ds = xr.open_dataset('/glade/work/afoster/FATES_calibration/observations/whittaker/whitkey.nc')

# out directory
out_dir = '/glade/work/afoster/FATES_calibration/ILAMB_testing'

# create a global land frac and area grid
land_frac_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")
target_grid = analysis.create_target_grid(land_frac_ds, 'FSR')

In [7]:
# FATES run with GSWP3
fates_gswp3_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_fates_sparse_vai_default_latest/lnd/hist'
fates_gswp3 = analysis.post_process_ds(fates_gswp3_dir, fates_vars, whittaker_ds,
                                     years=[2000, 2015], run_dict={'filter_nyears': 20})

In [8]:
# FATES sparse grid run - latest tag
fates_crujra_dir = '/glade/derecho/scratch/afoster/archive/ctsm60SP_fates_sparse_crujra/lnd/hist'
fates_crujra = analysis.post_process_ds(fates_crujra_dir, fates_vars, whittaker_ds,
                                     years=[2000, 2015], run_dict={'filter_nyears': 20})

In [19]:
target_grid

<xarray.DataArray 'FSR' (lat: 96, lon: 144)>
array([[111.57951 , 111.5852  , 111.595055, ..., 111.57626 , 111.57626 ,
        111.576805],
       [113.2382  , 113.41836 , 113.42712 , ..., 113.085175, 113.15016 ,
        113.0873  ],
       [112.15724 , 112.51087 , 112.586174, ..., 111.74958 , 111.82256 ,
        111.72557 ],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
  * lon        (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * lat        (lat) float32 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
    area       (lat, lon) float32 242.0 242.0 242.0 242.0 ... 0.0 0.0 0.0 0.0
    landmask   (lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    landfrac   (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0 0.0
    land_area  (lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [22]:
def get_data(ds, sparse_grid, var_list):
    ds_list = []
    for var in fates_vars:
        glob_ds = analysis.global_from_sparse(sparse_grid, ds[var], ds, ensemble=False).to_dataset(name=var)
        glob_ds[var].attrs = ds[var].attrs
        ds_list.append(glob_ds)
    ds_out = xr.merge(ds_list)

    # ds_out['landfrac'] = target_grid.landfrac
    # ds_out['landmask'] = target_grid.landmask
    # ds_out['area'] = target_grid.area
    # ds_out['land_area'] = target_grid.land_area    
    return ds_out

In [23]:
# gswp3 = get_data(fates_gswp3, sparse_grid, fates_vars)

In [ ]:
crujra = get_data(fates_crujra, sparse_grid, fates_vars)

In [ ]:
gswp3.to_netcdf(os.path.join(out_dir, 'hist_gswp3', 'gswp3_hist.nc'))
crujra.to_netcdf(os.path.join(out_dir, 'hist_crujra', 'crujra_hist.nc'))

In [24]:
crujra

<xarray.Dataset>
Dimensions:         (time: 192, lat: 96, lon: 144, levgrnd: 25)
Coordinates:
  * time            (time) object 2000-01-01 00:00:00 ... 2015-12-01 00:00:00
    gridcell        (lat, lon) int64 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
  * lat             (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon             (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
    area            (lat, lon) float32 242.0 242.0 242.0 242.0 ... 0.0 0.0 0.0
    landmask        (lat, lon) float64 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    landfrac        (lat, lon) float32 1.0 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0 0.0
    land_area       (lat, lon) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * levgrnd         (levgrnd) float32 0.01 0.04 0.09 0.16 ... 19.48 28.87 42.0
Data variables: (12/24)
    FATES_FRACTION  (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    FATES_GPP       (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    FATES_LAI       (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    QVEGE           (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    QVEGT           (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    QSOIL           (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    ...              ...
    FLDS            (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    FGR12           (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    TSA             (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    TREFMXAV        (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    TREFMNAV        (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan
    RH2M            (time, lat, lon) float32 nan nan nan nan ... nan nan nan nan